In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.layers import Dense, Flatten, Embedding, Activation, Dropout, LSTM, Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from tensorflow import string

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import sys
sys.path.append('/content/gdrive/My Drive')

In [5]:
!pip install unidecode

In [6]:
import data
data.get_data, data.clean_data

(<function data.get_data>, <function data.clean_data>)

In [7]:
df = data.get_data()
cleaned_df = data.clean_data(df)

In [21]:
cleaned_df

,id,text,target
0,1,deed reason earthquake may allah forgive u,1
1,4,forest fire near la canada,1
2,5,resident asked shelter place officer evacuatio...,1
3,6,people receive wildfire evacuation order calif...,1
4,7,got sent photo alaska smoke wildfire school,1
...,...,...,...
7608,10869,two giant crane holding bridge collapse nearby...,1
7609,10870,control wild fire california even northern par...,1
7610,10871,utckm volcano hawaii,1
7611,10872,police investigating ebike collided car little...,1


In [8]:
!pip install tensorflow-text==2.8.0rc0

In [9]:
import tensorflow_hub as hub
import tensorflow_text as text

In [10]:
seq_length = 25
batch_size = 32

In [11]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3") #arguments=dict(seq_length=seq_length))
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [44]:
#def build_classifier_model():
#    text_input = Input(shape=(), dtype=string, name='text')
#    preprocessing_layer = hub.KerasLayer(preprocess_url, name='preprocessing')
#    encoder_inputs = preprocessing_layer(text_input)
#    encoder = hub.KerasLayer(encoder_url, trainable=True, name='BERT_encoder')
#    outputs = encoder(encoder_inputs)
#     net = outputs['pooled_output']
#     net = Dropout(0.1)(net)
#   net = Dense(1, activation=None, name='classifier')(net)
#    return Model(text_input, net)

In [ ]:
#bert_model = build_classifier_model()
#bert_results = bert_model(cleaned_df['text'])

In [17]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [ ]:
#get_sentence_embeding(cleaned_df['text'])

In [12]:
text_input = Input(shape=(), dtype=string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [13]:
l = Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = Dense(1, activation='sigmoid', name="output")(l)

model = Model(inputs=[text_input], outputs = [l])

In [14]:
METRICS = [BinaryAccuracy(name='accuracy')]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [15]:
X = cleaned_df['text']
y = cleaned_df['target']

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [18]:
model.fit(X_train, y_train, epochs=10, batch_size=batch_size)

Epoch 1/10
179/179 [==============================] - 2660s 15s/step - loss: 0.6242 - accuracy: 0.6561
Epoch 2/10
179/179 [==============================] - 2640s 15s/step - loss: 0.5782 - accuracy: 0.7171
Epoch 3/10
179/179 [==============================] - 2645s 15s/step - loss: 0.5552 - accuracy: 0.7311
Epoch 4/10
179/179 [==============================] - 2655s 15s/step - loss: 0.5475 - accuracy: 0.7364
Epoch 5/10
179/179 [==============================] - 2648s 15s/step - loss: 0.5382 - accuracy: 0.7417
Epoch 6/10
179/179 [==============================] - 2647s 15s/step - loss: 0.5305 - accuracy: 0.7487
Epoch 7/10
179/179 [==============================] - 2657s 15s/step - loss: 0.5260 - accuracy: 0.7508
Epoch 8/10
179/179 [==============================] - 2664s 15s/step - loss: 0.5248 - accuracy: 0.7539
Epoch 9/10
149/179 [=======================>......] - ETA: 7:26 - loss: 0.5242 - accuracy: 0.7506

KeyboardInterrupt: ignored